<a href="https://www.kaggle.com/code/dbtmddn41/prompt-recovery-hf-lora-inference?scriptVersionId=173111230" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
%%capture
# If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/accelerate-0.27.2-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/transformers-4.38.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/optimum-1.17.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/utils-xla/peft-0.10.0-py3-none-any.whl --find-links=/kaggle/input/utils-xla

In [ ]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/llm-prompt-recovery"
    gemma = "/kaggle/input/gemma/transformers/1.1-7b-it/1"  # name of pretrained Gemma
    mistral = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
#     preset = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
    sequence_length = 1024 # max size of input sequence for training
    train_batch = 4 # size of the input batch in training
    validation_batch = 8
    epochs = 8 # number of epochs to train
    test_size = 0.
    train_datas = ['kishanvavdara', 'newtonbaba12345_3', 'newtonbaba12345_1', 'aatiffraz', 'newtonbaba12345_2', 'host']
    validation_datas = ["winddude"]
    lora_rank=4

In [ ]:
from accelerate.utils import BnbQuantizationConfig
from accelerate import Accelerator
import transformers
import optimum
import bitsandbytes
from peft import LoraConfig, get_peft_model, PeftModel

In [ ]:
import datetime
start_time = datetime.datetime.now()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

accelerator = Accelerator()

# Comment/Uncomment and use as per wish

# Found a good blog to catch me up fast!
# https://huggingface.co/blog/4bit-transformers-bitsandbytes
# https://huggingface.co/docs/transformers/v4.38.1/en/quantization#compute-data-type
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

gemma_tokenizer = AutoTokenizer.from_pretrained(CFG.gemma)
# tokenizer.padding_side = 'right'
gemma = AutoModelForCausalLM.from_pretrained(
    CFG.gemma,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
)

# model = model.to_bettertransformer()
# model = accelerator.prepare(model)

In [ ]:
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True,
# )

# mistral_tokenizer = AutoTokenizer.from_pretrained(CFG.mistral)
# mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
# mistral = AutoModelForCausalLM.from_pretrained(
#     CFG.mistral,
#     device_map = "auto",
#     trust_remote_code = True,
#     quantization_config=quantization_config,
# )

In [ ]:
# lora_config = LoraConfig(
#     r=CFG.lora_rank,
#     target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
#     task_type="CAUSAL_LM",
#     inference_mode=True,
#     lora_dropout=0.05,
#     bias="none",
# )
# peft_model = get_peft_model(model, lora_config)

In [ ]:
gemma=PeftModel.from_pretrained(gemma, "/kaggle/input/prompt-recovery-hf-tpu-train2/3-lora_gemmal7b-1.1")#, config=lora_config)#, device='cpu')

In [ ]:
import pandas as pd
from tqdm import tqdm

DEBUG = False

TEST_DF_FILE = '/kaggle/input/llm-prompt-recovery/test.csv'
SUB_DF_FILE = '/kaggle/input/llm-prompt-recovery/sample_submission.csv'
NROWS = None if DEBUG else None

if DEBUG:
    TEST_DF_FILE = '/kaggle/input/gemma-rewrite-nbroad/nbroad-v1.csv'
    SUB_DF_FILE = TEST_DF_FILE

tdf = pd.read_csv(TEST_DF_FILE, nrows=NROWS, usecols=['id', 'original_text', 'rewritten_text'])
sub = pd.read_csv(SUB_DF_FILE, nrows=NROWS, usecols=['id', 'rewrite_prompt'])

In [ ]:
val_df = pd.read_csv("/kaggle/input/70k-prompt-rewrite-triples/70k_gemma_template_built.csv")
val_df['rewritten_text']=val_df['generated_text']
val_df['id']=val_df.index

In [ ]:
def truncate_txt(text, length):
    text_list = text.split()
    
    if len(text_list) <= length:
        return text
    
    return " ".join(text_list[:length])


def gen_gemma_prompt(df):
    
    # Truncate the texts to first 200 words for now
    # As we are having memory issues on Mixtral8x7b
    og_text = truncate_txt(df["original_text"].strip(), CFG.sequence_length//3)
    rewritten_text = truncate_txt(df["rewritten_text"].strip(), CFG.sequence_length//3)
    template = """<start_of_turn>user\nOriginal Text:{}\nRewritten Text:{}\nWrite a prompt that was likely given to the LLM to rewrite original text into rewritten text.<end_of_turn>
<start_of_turn>model
Output:
"""
    return template.format(og_text, rewritten_text).strip()


In [ ]:
#original text prefix
orig_prefix = "Original Text:"

#mistral "response"
llm_response_for_rewrite = "Provide the new text and I will tell you what new element was added or change in tone was made to improve it - with no references to the original.  I will avoid mentioning names of characters.  It is crucial no person, place or thing from the original text be mentioned.  For example - I will not say things like 'change the puppet show into a book report' - I would just say 'improve this text into a book report'.  If the original text mentions a specific idea, person, place, or thing - I will not mention it in my answer.  For example if there is a 'dog' or 'office' in the original text - the word 'dog' or 'office' must not be in my response.  My answer will be a single sentence."

#modified text prefix
rewrite_prefix = "Re-written Text:"

#provided as start of Mistral response (anything after this is used as the prompt)
#providing this as the start of the response helps keep things relevant
response_start = "The request was: "

#added after response_start to prime mistral
#"Improve this" or "Improve this text" resulted in non-answers.  
#"Improve this text by" seems to product good results
response_prefix = "Improve this text by"

#well-scoring baseline text
#thanks to: https://www.kaggle.com/code/rdxsun/lb-0-61
base_line = 'Refine the following passage by emulating the writing style of [insert desired style here], with a focus on enhancing its clarity, elegance, and overall impact. Preserve the essence and original meaning of the text, while meticulously adjusting its tone, vocabulary, and stylistic elements to resonate with the chosen style.Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style.' 
examples_sequences = [
    (
        "Hey there! Just a heads up: our friendly dog may bark a bit, but don't worry, he's all bark and no bite!",
        "Warning: Protective dog on premises. May exhibit aggressive behavior. Ensure personal safety by maintaining distance and avoiding direct contact.",
        "Improve this text to be a warning."
    ),

    (
        "A lunar eclipse happens when Earth casts its shadow on the moon during a full moon. The moon appears reddish because Earth's atmosphere scatters sunlight, some of which refracts onto the moon's surface. Total eclipses see the moon entirely in Earth's shadow; partial ones occur when only part of the moon is shadowed.",
        "Yo check it, when the Earth steps in, takes its place, casting shadows on the moon's face. It's a full moon night, the scene's set right, for a lunar eclipse, a celestial sight. The moon turns red, ain't no dread, it's just Earth's atmosphere playing with sunlight's thread, scattering colors, bending light, onto the moon's surface, making the night bright. Total eclipse, the moon's fully in the dark, covered by Earth's shadow, making its mark. But when it's partial, not all is shadowed, just a piece of the moon, slightly furrowed. So that's the rap, the lunar eclipse track, a dance of shadows, with no slack. Earth, moon, and sun, in a cosmic play, creating the spectacle we see today.",
        "Improve this text to make it a rap."
    ),
    
    (
        "Drinking enough water each day is crucial for many functions in the body, such as regulating temperature, keeping joints lubricated, preventing infections, delivering nutrients to cells, and keeping organs functioning properly. Being well-hydrated also improves sleep quality, cognition, and mood.",
        "Arrr, crew! Sail the health seas with water, the ultimate treasure! It steadies yer body's ship, fights off plagues, and keeps yer mind sharp. Hydrate or walk the plank into the abyss of ill health. Let's hoist our bottles high and drink to the horizon of well-being!",
        "Improve this text to have a pirate."
    ),
    
    (
        "In a bustling cityscape, under the glow of neon signs, Anna found herself at the crossroads of endless possibilities. The night was young, and the streets hummed with the energy of life. Drawn by the allure of the unknown, she wandered through the maze of alleys and boulevards, each turn revealing a new facet of the city's soul. It was here, amidst the symphony of urban existence, that Anna discovered the magic hidden in plain sight, the stories and dreams that thrived in the shadows of skyscrapers.",
        "On an ordinary evening, amidst the cacophony of a neon-lit city, Anna stumbled upon an anomaly - a door that defied the laws of time and space. With the curiosity of a cat, she stepped through, leaving the familiar behind. Suddenly, she was adrift in the stream of time, witnessing the city's transformation from past to future, its buildings rising and falling like the breaths of a sleeping giant.",
        "Improve this text by making it about time travel."
    ),
    
    (
        "Late one night in the research lab, Dr. Evelyn Archer was on the brink of a breakthrough in artificial intelligence. Her fingers danced across the keyboard, inputting the final commands into the system. The lab was silent except for the hum of machinery and the occasional beep of computers. It was in this quiet orchestra of technology that Evelyn felt most at home, on the cusp of unveiling a creation that could change the world.",
        "In the deep silence of the lab, under the watchful gaze of the moon, Dr. Evelyn Archer found herself not alone. Beside her, the iconic red eye of HAL 9000 flickered to life, a silent partner in her nocturnal endeavor. 'Good evening, Dr. Archer,' HAL's voice filled the room, devoid of warmth yet comforting in its familiarity. Together, they were about to initiate a test that would intertwine the destiny of human and artificial intelligence forever. As Evelyn entered the final command, HAL processed the data with unparalleled precision, a testament to the dawn of a new era.",
        "Improve this text by adding an intelligent computer."
    ),
    
    (
        "The park was empty, save for a solitary figure sitting on a bench, lost in thought. The quiet of the evening was punctuated only by the occasional rustle of leaves, offering a moment of peace in the chaos of city life.",
        "Beneath the cloak of twilight, the park transformed into a realm of solitude and reflection. There, seated upon an ancient bench, was a lone soul, a guardian of secrets, enveloped in the serenity of nature's whispers. The dance of the leaves in the gentle breeze sang a lullaby to the tumult of the urban heart.",
        "Improve this text to be more poetic."
    ),
    
    (
        "The annual town fair was bustling with activity, from the merry-go-round spinning with laughter to the game booths challenging eager participants. Amidst the excitement, a figure in a cloak moved silently, almost invisibly, among the crowd, observing everything with keen interest but participating in none.",
        "Beneath the riot of color and sound that marked the town's annual fair, a solitary figure roamed, known to the few as Eldrin the Enigmatic. Clad in a cloak that shimmered with the whispers of the arcane, Eldrin moved with the grace of a shadow, his gaze piercing the veneer of festivity to the magic beneath. As a master of the mystic arts, he sought not the laughter of the crowds but the silent stories woven into the fabric of the fair. With a flick of his wrist, he could coax wonder from the mundane, transforming the ordinary into spectacles of shimmering illusion, his true participation hidden within the folds of mystery.",
        "Improve this text by adding a magician."
    ),
    
    (
        "The startup team sat in the dimly lit room, surrounded by whiteboards filled with ideas, charts, and plans. They were on the brink of launching a new app designed to make home maintenance effortless for homeowners. The app would connect users with local service providers, using a sophisticated algorithm to match needs with skills and availability. As they debated the features and marketing strategies, the room felt charged with the energy of creation and the anticipation of what was to come.",
        "In the quiet before dawn, a small group of innovators gathered, their mission: to simplify home maintenance through technology. But their true journey began with the unexpected addition of Max, a talking car with a knack for solving problems. 'Let me guide you through this maze of decisions,' Max offered, his dashboard flickering to life.",
        "Improve this text by adding a talking car."
    ),
]

def remove_numbered_list(text):
    final_text_paragraphs = []
    text = text.replace("  ", "\n")
    for line in text.split('\n'):
        # Split each line at the first occurrence of '. '
        parts = line.split('. ', 1)
        # If the line looks like a numbered list item, remove the numbering
        if len(parts) > 1 and parts[0].isdigit():
            final_text_paragraphs.append(parts[1])
        else:
            # If it doesn't look like a numbered list item, include the line as is
            final_text_paragraphs.append(line)

    return ' '.join(final_text_paragraphs)


#trims LLM output to just the response
def trim_to_response(text):
    terminate_string = "[/INST]"
    text = text.replace('</s>', '')
    #just in case it puts things in quotes
    text = text.replace('"', '')
    text = text.replace("'", '')

    last_pos = text.rfind(terminate_string)
    return text[last_pos + len(terminate_string):] if last_pos != -1 else text

#looks for response_start / returns only text that occurs after
def extract_text_after_response_start(full_text):
    parts = full_text.rsplit(response_start, 1)  # Split from the right, ensuring only the last occurrence is considered
    if len(parts) > 1:
        return parts[1].strip()  # Return text after the last occurrence of response_start
    else:
        return full_text  # Return the original text if response_start is not found

    
#trims text to requested number of sentences (or first LF or double-space sequence)
def trim_to_first_x_sentences_or_lf(text, x):
    if x <= 0:
        return ""

    # Any double-spaces dealt with as linefeed
    text = text.replace("  ", "\n")

    # Split text at the first linefeed
    text_chunks = text.split('\n', 1)
    first_chunk = text_chunks[0]

    # Split the first chunk into sentences, considering the space after each period
    sentences = [sentence.strip() for sentence in first_chunk.split('.') if sentence]

    # If there's a linefeed, return the text up to the first linefeed
    if len(text_chunks) > 1:
        # Check if the first chunk has fewer sentences than x, and if so, just return it
        if len(sentences) < x:
            trimmed_text = first_chunk
        else:
            # Otherwise, trim to x sentences within the first chunk
            trimmed_text = '. '.join(sentences[:x]).strip()
    else:
        # If there's no linefeed, determine if the number of sentences is less than or equal to x
        if len(sentences) <= x:
            trimmed_text = '. '.join(sentences).strip()  # Ensure space is preserved after periods
        else:
            # Otherwise, return the first x sentences, again ensuring space after periods
            trimmed_text = '. '.join(sentences[:x]).strip()

    # Add back the final period if it was removed and the text needs to end with a sentence.
    if len(sentences) > 0 and not trimmed_text.endswith('.'):
        trimmed_text += '.'

    return trimmed_text

def get_mistral_prompt(orig_text, transformed_text):

    messages = []

    # Append example sequences
    for example_text, example_rewrite, example_prompt in examples_sequences:
        messages.append({"role": "user", "content": f"{orig_prefix} {example_text}"})
        messages.append({"role": "assistant", "content": llm_response_for_rewrite})
        messages.append({"role": "user", "content": f"{rewrite_prefix} {example_rewrite}"})
        messages.append({"role": "assistant", "content": f"{response_start} {example_prompt}"})

    #actual prompt
    messages.append({"role": "user", "content": f"{orig_prefix} {orig_text}"})
    messages.append({"role": "assistant", "content": llm_response_for_rewrite})
    messages.append({"role": "user", "content": f"{rewrite_prefix} {transformed_text}"})
    messages.append({"role": "assistant", "content": f"{response_start} {response_prefix}"})
        
    #give it to Mistral
    model_inputs = mistral_tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = model_inputs.to("cuda") 
    generated_ids = mistral.generate(model_inputs, max_new_tokens=500, pad_token_id=mistral_tokenizer.eos_token_id)

    #decode and trim to actual response
    decoded = mistral_tokenizer.batch_decode(generated_ids)
    just_response = trim_to_response(decoded[0])        
    final_text = extract_text_after_response_start(just_response)
    
    #mistral has been replying with numbered lists - clean them up....
    final_text = remove_numbered_list(final_text)
    
    #mistral v02 tends to respond with the input after providing the answer - this tries to trim that down
    final_text = trim_to_first_x_sentences_or_lf(final_text, 1)
    #default to baseline if empty or unusually short
    if len(final_text) < 15:
        final_text = base_line
    
    return final_text

In [ ]:
import gc
import re

device = accelerator.device
tdf['id'] = sub['id'].copy()

pbar = tqdm(total=tdf.shape[0])

it = iter(tdf.iterrows())
idx, row = next(it, (None, None))

# https://www.kaggle.com/competitions/llm-prompt-recovery/discussion/481116
DEFAULT_TEXT = "Please improve this text using the writing style with maintaining the original meaning but altering the tone."

res = []

while idx is not None:
#     if idx > 40:
#         break
    if (datetime.datetime.now() - start_time) > datetime.timedelta(hours=8, minutes=30):
        res.append([row["id"], DEFAULT_TEXT])
        idx, row = next(it, (None, None))
        pbar.update(1)
        continue
        
    torch.cuda.empty_cache()
    gc.collect()
        
    try:   
        messages = gen_gemma_prompt(row)
        encoded_input = gemma_tokenizer(messages, return_tensors="pt", padding=True, truncation=True).to(device)

        with torch.no_grad():
            encoded_output = gemma.generate(**encoded_input, max_new_tokens=500)
            
        gemma_output = gemma_tokenizer.batch_decode(encoded_output[:, encoded_input.input_ids.shape[-1]:], skip_special_tokens=True)[0]
        if 'Prompt:' in gemma_output:
            gemma_output = gemma_output[gemma_output.find('Prompt:')+7:].lstrip(' \t\n*').split('\n')[0]
        elif 'Instructions:' in gemma_output:
            gemma_output = gemma_output[gemma_output.find('Instructions:')+13:].lstrip(' \t\n*').split('\n')[0]
        elif ':' in gemma_output:
            gemma_output = gemma_output.split(':')[1]
        gemma_output = gemma_output.strip(' \n\t*\"\'')
        if 15 > len(gemma_output):
            gemma_output = DEFAULT_TEXT
        
#         mistral_output = get_mistral_prompt(row['original_text'], row['rewritten_text'])
        output = gemma_output#+' '+mistral_output
        
        res.append([row["id"], output])
                            
    except Exception as e:
        print(f"ERROR: {e}")
        res.append([row["id"], DEFAULT_TEXT])
        
    finally:
        idx, row = next(it, (None, None))
        pbar.update(1)

        
pbar.close()


In [ ]:
sub = pd.DataFrame(res, columns=['id', 'rewrite_prompt'])

sub.to_csv("sample_submission.csv", index=False)
sub.to_csv("submission.csv", index=False)

In [ ]:
sub

In [ ]:
res